In [1]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarMDA
prob = om.Problem()
prob.model = SellarMDA()
prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-8)

prob.model.add_design_var('x', lower=0, upper=10)
prob.model.add_design_var('z', lower=0, upper=10)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0)
prob.model.add_constraint('con2', upper=0)

# Ask OpenMDAO to finite-difference across the model to compute the gradients for the optimizer
prob.model.approx_totals()

prob.setup()
print(prob['z'])

[5. 2.]


In [2]:
import ipywidgets as widgets
from openmdao.core.component import Component
from ipytree import Tree, Node
# from ipywidgets import Button, Label, FloatText, HBox, VBox, Text, Output, Layout
from ipywidgets import Label, FloatText, HBox, VBox, Output, Layout, Button


In [3]:
def set_vars_from_model(sys, node):
    if sys.name == '_auto_ivc':
        return
    name = sys.name if sys.name else 'root'
    new_node = Node(name)
    
    # can use any of these it looks like ! https://fontawesome.com/v5.15/icons?d=gallery&p=2&q=plus-&m=free
#     new_node.closed_icon = 'envelope'
#     new_node.open_icon = 'envelope-open'

    
    
    node.add_node(new_node)

    if isinstance(sys, Component):
        inputs = list(sys._var_allprocs_prom2abs_list['input'].keys())
        new_node.icon = 'plug'
        for input in inputs:
            input_node = Node(input)
            input_node.icon = 'signal'
            new_node.add_node(input_node)
    else:
        new_node.icon = 'envelope-open'
        for s in sys._subsystems_myproc:
            set_vars_from_model(s, new_node)

In [4]:
def set_vars_from_user_input(vars_to_set, node):
    for var in vars_to_set:
        val_widgets.append(var)
        val_widget = widgets.FloatText(
            value=7.5,
            description=var,
            disabled=False
        )
        val_widget.observe(update_prob_val, 'value')
        value_widget_box.children += (val_widget,)

In [5]:
def set_value_in_tree(node, name, value):
    current_node_name = node.name
    # just get the part before the colon
    actual_var_name = current_node_name.split(':')[0]
    if name == actual_var_name:
        node.name = f'{actual_var_name}: {value}'
        return
    for sub_node in node.nodes:
        set_value_in_tree(sub_node, name, value)

In [6]:
def update_prob_val(change):
    prob[change['owner'].description] = change['new']
    tree = widget.children[0]
#     set_value_in_tree(tree.nodes[0], change['owner'].description, change['new'] )
#     print(widget.children[0])
    # find the tree item that goes with this description and update the label with the value

In [17]:
output = Output(layout={'border': '1px solid black', 'width':'50%'})
val_widgets = []
value_widget_box = None

@output.capture()
def on_selected_change(change):
    print("change")
    global value_widget_box
#     print(change['new'][0], change['new'][0].selected)
#     if change['new'][0].selected:
#     print("change", change)
    change['new'][0].icon_style = 'warning'
    if change['new'][0].name in val_widgets:
        change['new'][0].selected = False
        return
    val_widgets.append(change['new'][0].name)
    val_widget = widgets.FloatText(
        value=7.5,
        description=change['new'][0].name,
        disabled=False
    )
    remove_button = Button(description="X")
    val_widget.observe(update_prob_val, 'value')
    val_and_remove_widget = HBox([val_widget, remove_button])
    value_widget_box.children += (val_and_remove_widget,)

In [18]:
@output.capture()
def select_vars_to_set(prob, vars_to_set = None):
    global value_widget_box
    tree = Tree(stripes=True, multiple_selection=True)
    value_widget_box = VBox([Label("Model Variables"),])
#     for n in tree.nodes:
#         print(n.name)
    if vars_to_set:
        set_vars_from_user_input(vars_to_set, tree)
    else:
        set_vars_from_model(prob.model, tree)
    tree.observe(on_selected_change, names='selected_nodes')
    widget = HBox([tree, value_widget_box, output])
    return widget

In [21]:
output.clear_output()

In [20]:
widget = select_vars_to_set(prob)
widget

In [11]:
prob['z']

array([5., 2.])

In [12]:
len(widget.children[0].nodes[0].nodes)

4

In [13]:
widget = select_vars_to_set(prob, vars_to_set=['x', 'y'])
widget

In [14]:
prob['x']

array([1.])

In [15]:
# # removing a widget
# remove = mybox.children[-1]
# mybox.children = mybox.children[:-1]
# remove.close()

# widgets.Checkbox(
#     value=False,
#     description='Check me',
#     disabled=False
# )

In [16]:
# def set_model_vars(prob, value_widget_box):
#     for var_widget in value_widget_box.children[3:]:
#         prob[var_widget.description] = var_widget.value